In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
train_set = pd.read_csv('/kaggle/input/titanic/train.csv')
test_set = pd.read_csv('/kaggle/input/titanic/test.csv')
gender_submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv', index_col=0)

train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_set.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
gender_submission.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


In [5]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
del train_set['Cabin']
del train_set['Ticket']
del train_set['Fare']

In [7]:
del test_set['Cabin']
del test_set['Ticket']
del test_set['Fare']

In [8]:
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,S


In [9]:
train_set['Name']=train_set['Name'].str.extract('([A-Za-z]+)\.')
pd.crosstab(train_set['Name'], train_set['Sex'])

Sex,female,male
Name,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [10]:
train_set['Name'] = train_set['Name'].replace(['Capt', 'Col', 'Countess', 'Don', 'Dr', 'Jonkheer', 'Lady', 'Major', 'Rev', 'Sir'], 'Others')
train_set['Name'] = train_set['Name'].replace('Mlle', 'Miss')
train_set['Name'] = train_set['Name'].replace('Mme', 'Mrs')
train_set['Name'] = train_set['Name'].replace('Ms', 'Miss')

train_set['Name'].value_counts()

Mr        517
Miss      185
Mrs       126
Master     40
Others     23
Name: Name, dtype: int64

In [11]:
train_set['Name'] = train_set['Name'].replace({'Mr': 0, 'Miss': 1, 'Mrs':2, 'Master':3, 'Others':4})
train_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,1,0,3,0,male,22.0,1,0,S
1,2,1,1,2,female,38.0,1,0,C
2,3,1,3,1,female,26.0,0,0,S
3,4,1,1,2,female,35.0,1,0,S
4,5,0,3,0,male,35.0,0,0,S


In [12]:
train_set = train_set.replace(['male', 'female'], [1, 2])
train_set['Embarked'] = train_set['Embarked'].replace(['S', 'C', 'Q', np.NAN], [1, 2, 3, 1])

train_set

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,1,0,3,0,1,22.0,1,0,1
1,2,1,1,2,2,38.0,1,0,2
2,3,1,3,1,2,26.0,0,0,1
3,4,1,1,2,2,35.0,1,0,1
4,5,0,3,0,1,35.0,0,0,1
...,...,...,...,...,...,...,...,...,...
886,887,0,2,4,1,27.0,0,0,1
887,888,1,1,1,2,19.0,0,0,1
888,889,0,3,1,2,NaN,1,2,1
889,890,1,1,0,1,26.0,0,0,2


In [13]:
y_data = train_set['Survived']

x_data = train_set.copy()
del x_data['Survived']

x_data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,1,3,0,1,22.0,1,0,1
1,2,1,2,2,38.0,1,0,2
2,3,3,1,2,26.0,0,0,1
3,4,1,2,2,35.0,1,0,1
4,5,3,0,1,35.0,0,0,1
...,...,...,...,...,...,...,...,...
886,887,2,4,1,27.0,0,0,1
887,888,1,1,2,19.0,0,0,1
888,889,3,1,2,NaN,1,2,1
889,890,1,0,1,26.0,0,0,2


In [14]:
x_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Name           891 non-null int64
Sex            891 non-null int64
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Embarked       891 non-null int64
dtypes: float64(1), int64(7)
memory usage: 55.8 KB


In [15]:
temp = x_data.copy()
del temp['Age']

from sklearn import cluster
import math

kmeans = cluster.KMeans(n_clusters=5, random_state=0).fit(temp)
x_data['Label'] = kmeans.labels_

age_sum = [0.0,0.0,0.0,0.0,0.0]
age_num = [0.0,0.0,0.0,0.0,0.0]

for i, label in enumerate(x_data['Label']):
    if math.isnan(x_data['Age'][i]):
        continue
    else:
        age_sum[label] += x_data['Age'][i]
        age_num[label] += 1
        
# avg
age_mean = [0.0, 0.0, 0.0, 0.0, 0.0]
for i in range(5):
    age_mean[i] = age_sum[i]/age_num[i]
    
# fill age
for i, label in enumerate(x_data['Label']):
    if math.isnan(x_data['Age'][i]):
        x_data['Age'][i] = age_mean[label]
        
        

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
x_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    891 non-null int64
Pclass         891 non-null int64
Name           891 non-null int64
Sex            891 non-null int64
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Embarked       891 non-null int64
Label          891 non-null int32
dtypes: float64(1), int32(1), int64(7)
memory usage: 59.3 KB


In [17]:
# train - test split
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler

x_train, x_test, y_train, y_test = model_selection.train_test_split(x_data, y_data, test_size=0.2, random_state=0)

# use standardscaler
scaler = StandardScaler()
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [18]:
x_train_scaled

array([[-1.16343003,  0.81925059,  1.30164644, ...,  1.95926403,
         0.99748327,  0.69868925],
       [-0.01263834, -0.38096838, -0.70218548, ..., -0.47741019,
        -0.56341363,  1.41139232],
       [ 1.44220868, -0.38096838, -0.70218548, ...,  0.74092692,
         0.99748327, -0.01401382],
       ...,
       [ 0.71863397,  0.81925059, -0.70218548, ..., -0.47741019,
         2.55838016, -0.7267169 ],
       [ 0.44921786,  0.81925059,  1.30164644, ..., -0.47741019,
        -0.56341363, -0.7267169 ],
       [ 0.93031806, -0.38096838, -0.70218548, ...,  0.74092692,
        -0.56341363, -0.7267169 ]])

In [19]:
# select model 
# xgboost, RandomForest, GradientBoosting
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier

model1 = XGBClassifier()
model2 = RandomForestClassifier()
model3 = GradientBoostingClassifier()
model4 = CatBoostClassifier()

In [20]:
# fit(train)
model1.fit(x_train_scaled, y_train)
model2.fit(x_train_scaled, y_train)
model3.fit(x_train_scaled, y_train)
model4.fit(x_train_scaled, y_train)


# fit(test)
model1.fit(x_test_scaled, y_test)
model2.fit(x_test_scaled, y_test)
model3.fit(x_test_scaled, y_test)
model4.fit(x_test_scaled, y_test)


# score_test 
score_test1 = model1.score(x_train_scaled, y_train)
print('xgboost : {:3f}'.format(score_test1))

score_test2 = model1.score(x_train_scaled, y_train)
print('RandomForest : {:3f}'.format(score_test2))

score_test3 = model1.score(x_train_scaled, y_train)
print('GradientBoosting : {:3f}'.format(score_test3))

score_test4 = model1.score(x_train_scaled, y_train)
print('CatBoost : {:3f}'.format(score_test4))


Learning rate set to 0.008911
0:	learn: 0.6871584	total: 57.5ms	remaining: 57.4s
1:	learn: 0.6817002	total: 59ms	remaining: 29.4s
2:	learn: 0.6765845	total: 60.3ms	remaining: 20s
3:	learn: 0.6709861	total: 61.7ms	remaining: 15.4s
4:	learn: 0.6655575	total: 63ms	remaining: 12.5s
5:	learn: 0.6609103	total: 64.3ms	remaining: 10.6s
6:	learn: 0.6557050	total: 65.6ms	remaining: 9.31s
7:	learn: 0.6506094	total: 67.1ms	remaining: 8.32s
8:	learn: 0.6465965	total: 67.9ms	remaining: 7.48s
9:	learn: 0.6411615	total: 69.2ms	remaining: 6.85s
10:	learn: 0.6371962	total: 70.6ms	remaining: 6.34s
11:	learn: 0.6326042	total: 71.9ms	remaining: 5.92s
12:	learn: 0.6280456	total: 73.1ms	remaining: 5.55s
13:	learn: 0.6236962	total: 74.5ms	remaining: 5.25s
14:	learn: 0.6196229	total: 75.9ms	remaining: 4.98s
15:	learn: 0.6152273	total: 77.3ms	remaining: 4.75s
16:	learn: 0.6114584	total: 78.2ms	remaining: 4.52s
17:	learn: 0.6082287	total: 79.7ms	remaining: 4.34s
18:	learn: 0.6042922	total: 81ms	remaining: 4.18s


In [21]:
# real test
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Embarked       418 non-null object
dtypes: float64(1), int64(4), object(3)
memory usage: 26.2+ KB


In [22]:
test_set['Name'].value_counts()

Ashby, Mr. John                                     1
Hipkins, Mr. William Edward                         1
Lahtinen, Rev. William                              1
Moubarek, Mrs. George (Omine Amenia" Alexander)"    1
Abrahamsson, Mr. Abraham August Johannes            1
                                                   ..
Laroche, Miss. Louise                               1
Sage, Miss. Ada                                     1
Gibson, Miss. Dorothy Winifred                      1
Borebank, Mr. John James                            1
Maybery, Mr. Frank Hubert                           1
Name: Name, Length: 418, dtype: int64

In [23]:
# name        
test_set['Name']=test_set['Name'].str.extract('([A-Za-z]+)\.')
pd.crosstab(test_set['Name'], test_set['Sex'])

test_set['Name'] = test_set['Name'].replace(['Capt', 'Col', 'Countess', 'Don','Dona', 'Dr', 'Jonkheer', 'Lady', 'Major', 'Rev', 'Sir'], 'Others')
test_set['Name'] = test_set['Name'].replace('Mlle', 'Miss')
test_set['Name'] = test_set['Name'].replace('Mme', 'Mrs')
test_set['Name'] = test_set['Name'].replace('Ms', 'Miss')

test_set['Name'] = test_set['Name'].replace({'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Others':4})

# sex, embarked
test_set = test_set.replace(['male', 'female'], [1, 2])
test_set['Embarked'] = test_set['Embarked'].replace(['S', 'C', 'Q', np.NAN], [1, 2, 3, 1])

test_set.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,892,3,0,1,34.5,0,0,3
1,893,3,2,2,47.0,1,0,1
2,894,2,0,1,62.0,0,0,3
3,895,3,0,1,27.0,0,0,1
4,896,3,2,2,22.0,1,1,1


In [24]:
test_set['Name'].value_counts()

0    240
1     79
2     72
3     21
4      6
Name: Name, dtype: int64

In [25]:
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null int64
Sex            418 non-null int64
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Embarked       418 non-null object
dtypes: float64(1), int64(6), object(1)
memory usage: 26.2+ KB


In [26]:
# test_set['Fare'] = test_set['Fare'].fillna(test_set['Fare'].mean())
# test_set.info() 생략

In [27]:
test_set['Embarked'].value_counts()
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null int64
Sex            418 non-null int64
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Embarked       418 non-null object
dtypes: float64(1), int64(6), object(1)
memory usage: 26.2+ KB


In [28]:
# age

temp = test_set.copy()
del temp['Age']

from sklearn import cluster
import math

kmeans = cluster.KMeans(n_clusters=5, random_state=0).fit(temp)
test_set['Label'] = kmeans.labels_

age_sum = [0.0,0.0,0.0,0.0,0.0]
age_num = [0.0,0.0,0.0,0.0,0.0]

for i, label in enumerate(test_set['Label']):
    if math.isnan(test_set['Age'][i]):
        continue
    else:
        age_sum[label] += test_set['Age'][i]
        age_num[label] += 1
        
# avg
age_mean = [0.0, 0.0, 0.0, 0.0, 0.0]
for i in range(5):
    age_mean[i] = age_sum[i]/age_num[i]
    
# fill age
for i, label in enumerate(test_set['Label']):
    if math.isnan(test_set['Age'][i]):
        test_set['Age'][i] = age_mean[label]
        
test_set.info()

    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null int64
Sex            418 non-null int64
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Embarked       418 non-null object
Label          418 non-null int32
dtypes: float64(1), int32(1), int64(6), object(1)
memory usage: 27.9+ KB


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [29]:
real_test_scaled = scaler.transform(test_set)

In [30]:
pred_test1 = model1.predict(real_test_scaled)
pred_test2 = model2.predict(real_test_scaled)
pred_test3 = model3.predict(real_test_scaled)
pred_test4 = model4.predict(real_test_scaled)



In [31]:
submission = pd.DataFrame(data=pred_test1, columns=gender_submission.columns, index=gender_submission.index)
submission.to_csv('/kaggle/working/submission_data1.csv', index=True)

submission = pd.DataFrame(data=pred_test2, columns=gender_submission.columns, index=gender_submission.index)
submission.to_csv('/kaggle/working/submission_data2.csv', index=True)

submission = pd.DataFrame(data=pred_test3, columns=gender_submission.columns, index=gender_submission.index)
submission.to_csv('/kaggle/working/submission_data3.csv', index=True)

submission = pd.DataFrame(data=pred_test4, columns=gender_submission.columns, index=gender_submission.index)
submission.to_csv('/kaggle/working/submission_data4.csv', index=True)
